# CNN Forward and Backward(Theory01)

为了方便理解,我们直接使用例子来讲解CNN的Forward Propagation和Backward Propagation.

我们以CNN(Theory)中的模型为例子(Single Sample):

<img src="../../picture/156.png" width=700 height=700>

我们一般将CNN划分成两部分:1.CONV layers,2.FC layers.其中FC layers和我们DNN中是一样的.接下去我们按照模型走一次FP和BP.

## Forward Propagation

**[1] CONV**



对于卷积层的矩阵乘法实现:卷积运算本质上就是在卷积核(滤波器)和输入数据的局部区域间做**点积**,我们常常使用**im2col**进行运算卷积.

**im2col:**

假设我们有的输入是(4x4),卷积操作$f=3x3,s=1,padding=0,\#filters=1$,那么最终的结果是($\frac{4+2*0-3}{1}+1=2$).

那么我们会将卷积核滑过的区域依次展平(```X_col```)保存,最终得(9x4)的矩阵.同样将卷积核(```W_row```)也展平(1x9).继而再将两者做```np.dot(W_row, X_col)```,得到的结果和卷积核在每一个位置的输出是等价的.

<img src="../../picture/162.png" width=700 height=700>

<span style="color:orange">需要注意的是,最后输出的结果(1x4)需要合理转换为(2x2).</span>


**Ps:**
> 1.至于将感受野转换成列还是行没有多大区别,caffe框架是展开为行,MATLAB是列.

> 2.优点:计算机一次性读取相近的内存是最快的,尤其是当需要把数据送到GPU去计算的时候,这样可以节省访存的时间,以达到加速的目的.如果是按照原来的方式其是断续的,效率会变低.

> 3.缺点:占用内存太多,因为在输入数据体中的某些值在X_col中被复制了多次,虽然同一个位置的数据可能被多次卷积,但是数字却是同一个数字,那么多次复制实际上是一个浪费内存的过程.

所以整体的CONV表示为:

$Z^{[l]}=A^{[l-1]}"*" W^{[l]}+b^{[l]}$

$A^{[l]}=activation(Z^{[l]})$

其中"*"表示卷积操作,这里我们使用im2col.

**[2] POOL**

对于CONV1到POOL1就比较简单了,根据提供的$f,s$进行获取最大值.

**[3] FC**

最终将卷积层的结果展平进入全连接层.全连接层和之前的DNN完全一样,这里就不在多说了.

**Ps:**

> 现在越来越多是将全连接层转换成全卷机层以便适应更多情况,具体情况我们在目标检测中详细讨论.

## Backward Propagation

回顾DNN(Single Sample)中我们的BP如下:

$dZ^{[l]} = dA^{[l]} * g^{[l]^{'}}(Z^{[l]})$

$dW^{[l]} =  dZ^{[l]} A^{[l-1]}$

$db^{[l]} = dZ^{[l]}$

$dA^{[l-1]} = W^{{[l]}^{T}} dZ^{[l]}$

那么我们可以使用DNN的BP套用在CNN中,但是我们需要解决几个问题:

对于CNN而言,FC是和DNN一样的,可以直接套用DNN的BP,但是对于CONV层而言:

[1] Pool层没有任何参数,也没有激活函数,也就意味着我们令$\sigma(Z)=Z$,即激活后就是本身,那么Pool层的导数就为1.

[2] Pool层在FP的时候对输入进行了压缩,那么我们在BP的时候需要还原到原来的大小.

[3] 卷积层虽然和FC一样是在做点积操作,但是其只是在感受野内,所以就和DNN很不一样.

[4] 对于卷积层,由于$W$是使用卷积,那么BP推导出的$W,b$方式也就不同.

**至此,针对于以上问题,我们开始逐步推导CNN的BP.需要注意的是CONV中可以采用多个卷积核,各个卷积核完全相同独立,为了简化公式的复杂度,我们这里就使用若干卷积核中的一个.**

### 1. 已知池化层的${\color{Red} {dA^{[l]}}}$求上一层的${\color{Purple} {dA^{[l]}}}$

注意:由于我在FP中写的是,${\color{Red} {A^{[2]}}}=Pool2(P^{[2]},{\color{Purple} {A^{[2]}}})$所以实际上就是已知红色部分${\color{Red} {dA^{[2]}}}$求紫色部分的${\color{Purple} {dA^{[2]}}}$

在FP时,池化层一般我们会用MAX或者Average对输入进行池化,池化的区域大小已知.现在我们反过来,要从缩小后的误差$dA^{[l+1]}$,还原前一次较大区域对应的误差.

在反向传播时:

1) 首先会把$dA^{[l+1]}$的所有子矩阵矩阵大小还原成池化之前的大小.

2) 如果是MAX,则把$dA^{[l+1]}$的所有子矩阵的各个池化局域的值放在之前做前向传播算法得到**最大值的位置**.

3) 如果是Average,则把$dA^{[l+1]}$的所有子矩阵的各个池化局域的值**取平均后放在还原后的子矩阵位置**.

这个过程一般叫做上采样(Upsample)

举一个简单的例子:

池化层($f=2,s=2$)

**MAX:**
<img src="../../picture/163.png" width=700 height=700>

**Ps:**

注意这里为了演示才将BP中的值与FP输出的值令成相同.实际上其是不同的.

方法处理:这里方法处理有很多可以实现,包括

- (1:推荐)正向传播的时候就缓存最大值的位置(一般指的索引位置),然后在方法处理中直接将池化层后的结果放入对应的位置,其余补零.

- (2)就是使用下面的方式:

> 我们应当先创建一个mask矩阵,这里矩阵为每个分割输入矩阵中最大位置例如:
$X=\begin{bmatrix}
1 &3 \\ 
 4&2 
\end{bmatrix} \rightarrow M=\begin{bmatrix}
0 &0 \\ 
 1&0 
\end{bmatrix}$

> 然后再将池化后得到的结果元素相乘mask,放入对应的分割范围位置
 
> <img src="../../picture/165.png" width=700 height=700>

**Average:**

<img src="../../picture/164.png" width=700 height=700>

这样我们就可以知道${\color{Purple} {dA^{[l]}}}$,进而:

$dZ^{[l]} = {\color{Purple} {dA^{[l]}}} * g^{[l]^{'}}(Z^{[l]})$

也就是求的了卷积层卷积后的梯度.

### 2. 已知卷积层的$dZ^{[l+1]}$求上一层的$dA^{[l]}$

首先我们来回顾一下卷积层的FP:

$Z^{[l]}=CONV2(W^{[l]},A^{[l-1]})+b^{[l]}$

那么在DNN中,我们的$dA^{[l-1]} = W^{{[l]}^{T}} dZ^{[l]}$.

那么类似的,我们的卷积层中的:

$dA^{[l]}=dZ^{[l+1]} "*" rot180(W^{{[l+1]}})$

或者:

$dA^{[l]}=W^{[l+1]} "*" rot180(dZ^{{[l+1]}})$

其中$"*"$表示卷积操作

<span style="color:orange">那么与DNN相比,卷积层的$dA^{[l]}$中$W^{[l+1]}$被逆时针旋转了180°,而在DNN中仅仅是转置.那么为什么卷积核要逆时针旋转180°呢?</span>

我们来举一个例子就知道了.

假设$l-1$层的输出$A^{[l-1]}$是一个3x3矩阵,第$l$层的卷积核$W^{[l]}$是一个2x2矩阵,采用1像素的步幅,则输出$z^{[l]}$是一个2x2的矩阵.我们简化都是$b^{[l]}$都是0,则有

$A^{[l-1]}"*"W^{[l]}=Z^{[l]}$

列出矩阵表达式则为:



$\begin{bmatrix}
a_{11} &a_{12}  &a_{13} \\ 
a_{21} &  a_{22}&a_{23} \\ 
 a_{31}&a_{32}  & a_{33}
\end{bmatrix} "*"\begin{bmatrix}
w_{11} & w_{12} \\ 
 w_{21} & w_{22} 
\end{bmatrix}=\begin{bmatrix}
z_{11} & z_{12}\\ 
 z_{21}& z_{22}
\end{bmatrix}$

由卷积定义可以得出:

$z_{11} = a_{11}w_{11}+a_{12}w_{12}+a_{21}w_{21}+a_{22}w_{22}$

$z_{12} = a_{12}w_{11}+a_{13}w_{12}+a_{22}w_{21}+a_{23}w_{22}$

$z_{21} = a_{21}w_{11}+a_{22}w_{12}+a_{31}w_{21}+a_{32}w_{22}$

$z_{22} = a_{22}w_{11}+a_{23}w_{12}+a_{32}w_{21}+a_{33}w_{22}$

接着我们模拟反向推导:

### $da^{[l-1]}=dz^{[l]}\frac{\partial{z^{[l]}}}{\partial{a^{[l-1]}}}$


可以看到对于$a^{[l-1]}$的梯度$da^{[l-1]}$等于$dz^{[l]}$从乘以$\frac{\partial{z^{[l]}}}{\partial{a^{[l-1]}}}$,而$\frac{\partial{z^{[l]}}}{\partial{a^{[l-1]}}}$实际上是一个关于$w$的值.

假设我们的$z^{[l]}$矩阵的反向传播值为$dz_{11},dz_{12},dz_{21},dz_{22}$组成的(2x2)的矩阵,那么利用上面的4个等式,我们可以分别写出$da^{[l-1]}$的9个导数:

比如对于第一个$a_{11}$的导数$da_{11}$,在4个等式中$a_{11}$只与$z_{11}$有关,从而我们有:

$da_{11}=dz_{11}w_{11}$

对于$a_{12}$的梯度,上面4个等式中和$a_{12}$有关的只有$z_{12},z_{11}$,从而我们有:

$da_{12}=dz_{11}w_{12}+dz_{12}w_{11}$

同理我们可以得到:

$da_{13}=dz_{12}w_{12}$

$da_{21}=dz_{11}w_{21}+dz_{21}w_{11}$

$da_{22}=dz_{11}w_{22}+dz_{12}w_{21}+dz_{21}w_{12}+dz_{22}w_{11}$

$da_{23}=dz_{12}w_{22}+dz_{22}w_{12}$

$da_{31}=dz_{21}w_{21}$

$da_{32}=dz_{21}w_{22}+dz_{22}w_{21}$

$da_{33}=dz_{22}w_{22}$

其实上面的9个导数可以写成卷积矩阵的形式:

$\begin{bmatrix}
0 &0  &0  &0 \\ 
 0& dz_{11} &dz_{12}  &0 \\ 
 0&  dz_{21}&dz_{22}  &0 \\ 
 0&  0&  0& 0
\end{bmatrix}"*"\begin{bmatrix}
w_{22} &w_{21} \\ 
 w_{12}& w_{11}
\end{bmatrix}=\begin{bmatrix}
da_{11} &da_{12}  &da_{13} \\ 
 da_{21}&  da_{22}& da_{23}\\ 
 da_{31}&  da_{32}& da_{33}
\end{bmatrix}$

为了符合梯度计算,在$dz$矩阵周围填充了一圈0,此时将卷积核翻转后和反向传播的梯度误差进行卷积,就得到了前一次的梯度误差.这个例子直观的介绍了为什么对含有卷积的式子求导时,卷积核要rot180°度的原因,因为:

$rot180\left (  \begin{bmatrix}
w_{11} & w_{12} \\ 
 w_{21} & w_{22} 
\end{bmatrix}\right )=\begin{bmatrix}
w_{22} &w_{21} \\ 
 w_{12}& w_{11}
\end{bmatrix}$

**同样:**

$\begin{bmatrix}
0 &0  &0  &0 \\ 
 0& w_{11} &w_{12}  &0 \\ 
 0&  w_{21}&w_{22}  &0 \\ 
 0&  0&  0& 0
\end{bmatrix}"*"\begin{bmatrix}
dz_{22} &dz_{21} \\ 
 dz_{12}& dz_{11}
\end{bmatrix}=\begin{bmatrix}
da_{11} &da_{12}  &da_{13} \\ 
 da_{21}&  da_{22}& da_{23}\\ 
 da_{31}&  da_{32}& da_{33}
\end{bmatrix}$

所以:

$dA^{[l]}=dZ^{[l+1]} "*" rot180(W^{{[l+1]}})$

或者:

$dA^{[l]}=W^{[l+1]} "*" rot180(dZ^{{[l+1]}})$

**Ps:**

> 由于我们知道$\left \lfloor \frac{n+2p-f}{s} +1 \right \rfloor \times \left \lfloor \frac{n+2p-f}{s} +1 \right \rfloor $

> 那么:

> $\frac{n_{input}+2p-f}{s} +1=n_{out}$,由于$n_{input},f,s,n_{out}$都是已知,我们就可以计算出在BP的时候外层padding的值.以上面的例子为例:

> $\frac{2+2p-2}{1}+1=3 \rightarrow p=1$


### 3. 已知卷积层的$dZ^{[l+1]}$求该层的$dW^{[l]},db^{[l]}$

到现在已经可以递推出每一层的$dZ^{[l+1]}$了,对于全连接层,可以按照DNN的流程,池化层没有$W,b$所以不需要求梯度,卷积层的$W,b$如下:

对于FP:$Z^{[l]}=CONV(W^{[l]},A^{[l-1]})+b^{[l]}$

再来看看DNN中的$dW$:$dW^{[l]} =  dZ^{[l]} A^{[l-1]}$

因此在CONV中$dW:$$dW^{[l]}=rot180(dZ^{[l]}"*"A^{[l-1]})$或者$dW^{[l]}=A^{[l-1]}"*"dZ^{[l]}$

有了上面的基础,我们就知道为什么这里需要$rot180$了,我们依然使用上面的例子来看看:

**FP:**

$\begin{bmatrix}
a_{11} &a_{12}  &a_{13} \\ 
a_{21} &  a_{22}&a_{23} \\ 
 a_{31}&a_{32}  & a_{33}
\end{bmatrix} "*"\begin{bmatrix}
w_{11} & w_{12} \\ 
 w_{21} & w_{22} 
\end{bmatrix}=\begin{bmatrix}
z_{11} & z_{12}\\ 
 z_{21}& z_{22}
\end{bmatrix}$

那么由卷积的定义可以得出:

$z_{11} = a_{11}w_{11}+a_{12}w_{12}+a_{21}w_{21}+a_{22}w_{22}$

$z_{12} = a_{12}w_{11}+a_{13}w_{12}+a_{22}w_{21}+a_{23}w_{22}$

$z_{21} = a_{21}w_{11}+a_{22}w_{12}+a_{31}w_{21}+a_{32}w_{22}$

$z_{22} = a_{22}w_{11}+a_{23}w_{12}+a_{32}w_{21}+a_{33}w_{22}$


那么接下去模拟反向梯度:

 $dw^{[l]}=dz^{[l]}\frac{\partial{z^{[l]}}}{\partial{w^{[l]}}}$

很明显,$\frac{\partial{z^{[l]}}}{\partial{w^{[l]}}}$是与$a$有关的值,那么:

$dw_{11}=dz_{11}a_{11}+dz_{12}a_{12}+dz_{21}a_{21}+dz_{22}a_{22}$

$dw_{12}=dz_{11}a_{12}+dz_{12}a_{13}+dz_{21}a_{22}+dz_{22}a_{23}$

$dw_{21}=dz_{11}a_{21}+dz_{12}a_{22}+dz_{21}a_{31}+dz_{22}a_{32}$

$dw_{22}=dz_{11}a_{22}+dz_{12}a_{23}+dz_{21}a_{32}+dz_{22}a_{33}$

那么用矩阵表示即:

$\begin{bmatrix}
0 &0  &0  &0 \\ 
 0& dz_{11} &dz_{12}  &0 \\ 
 0&  dz_{21}&dz_{22}  &0 \\ 
 0&  0&  0& 0
\end{bmatrix}"*"\begin{bmatrix}
da_{11} &da_{12}  &da_{13} \\ 
 da_{21}&  da_{22}& da_{23}\\ 
 da_{31}&  da_{32}& da_{33}
\end{bmatrix}=\begin{bmatrix}
dw_{22} &dw_{21} \\ 
 dw_{12}& dw_{11}
\end{bmatrix}$

**同样:**

我们可以发现如果使用:

$\begin{bmatrix}
a_{11} &a_{12}  &a_{13} \\ 
a_{21} &  a_{22}&a_{23} \\ 
 a_{31}&a_{32}  & a_{33}
\end{bmatrix} "*"\begin{bmatrix}
dz_{11} & dz_{12}\\ 
 dz_{21}& dz_{22}
\end{bmatrix}=\begin{bmatrix}
dw_{11} & dw_{12} \\ 
 dw_{21} & dw_{22} 
\end{bmatrix}$

这样子结果就不需要$rot180$,实际上就和DNN中转置的思想是一样的.

对于$db^{[l]}$而言,我们需要保证通道是与$dZ^{[l]}$是一致的,所以我们会将每一个通道上的$dZ_{row,col}$累和:

$db^{[l]}=\sum_{row,col}Z^{[l]}_{row,col}$

也就是如果是多卷积核,最终的shape:$(1\times 1\times channels )$

**Reference:**

[1] [深度学习之CNN反向传播算法详解](https://cloud.tencent.com/developer/article/1087188)

[2] [im2col的原理和实现](https://blog.csdn.net/dwyane12138/article/details/78449898)

[3] [卷积神经网络正反向传播算法总结](https://zhuanlan.zhihu.com/p/33876102)